In [1]:
%run "../layeringOfServices.ipynb"

In [21]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd

In [ ]:
from re import sub
def name_handler(s):
    s = sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespace with a single dash
    s = sub(r"\s+", '_', s)
    return s



## Old hts communes 

hts_5Mai2021 = pd.read_excel("hts_5Mai2021.xlsx")

hts_5Mai2021.date_entevyou.min().strftime("%Y-%m-%d")
hts_5Mai2021.date_entevyou.max().strftime("%Y-%m-%d")

hts_5Mai2021.actual_commune.unique()

hts_5Mai2021_communes = []
for columns in hts_5Mai2021.actual_commune.unique():
    globals()[f"hts5Juin_{name_handler(columns)}"] = hts_5Mai2021[hts_5Mai2021.actual_commune == columns]
    hts_5Mai2021_communes.append(globals().get(f"hts5Juin_{name_handler(columns)}"))

communes_names_General = hts_5Mai2021.actual_commune.unique().tolist()
hts_5Mai2021_dict = {}
for columns, communes in zip(hts_5Mai2021.actual_commune.unique(), communes_names_General):
    hts_5Mai2021_dict[f"{name_handler(communes)}"] =  globals().get(f"hts5Juin_{name_handler(columns)}").code.count()
    
    
hts_cincoMayo_df = pd.DataFrame.from_dict(hts_5Mai2021_dict, orient="index", columns=None)
hts_cincoMayo_df.columns = ["nombre_filles_pour_depistage_hts"]
hts_cincoMayo_df

liste_hts_5Mai2021 = pd.ExcelWriter("rapportage_liste_filles_hts_5Mai2021.xlsx",engine="xlsxwriter")
hts_cincoMayo_df.to_excel(liste_hts_5Mai2021,sheet_name="Total_filles_pr_commune",index=True)
hts_5Mai2021.to_excel(liste_hts_5Mai2021,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,communes in zip(hts_5Mai2021.actual_commune.unique(), communes_names_General):
    globals().get(f"hts5Juin_{name_handler(columns)}").to_excel(liste_hts_5Mai2021,sheet_name=f"{name_handler(communes)}",index=False,na_rep="NULL")
liste_hts_5Mai2021.save()


## Nouvelles Filles HTS

In [90]:
hts_new = dreams_mastersheet[(dreams_mastersheet.timeOn_system == "required_Time_on")&(dreams_mastersheet.hts == "errata")&(dreams_mastersheet.newage_range != "10-17")&(dreams_mastersheet.newage_range != "25-29")&(dreams_mastersheet.newage_range != "not_valid_age")]

hts_new.code.count()


4799

hts_5Mai2021.code.count()

hts_5Mai2021.date_entevyou.min().strftime("%Y-%m-%d")
hts_5Mai2021.date_entevyou.max().strftime("%Y-%m-%d")


In [93]:
hts_new.date_entevyou.min().strftime("%Y-%m-%d")
hts_new.date_entevyou.max().strftime("%Y-%m-%d")

'2020-04-07'

'2021-06-15'

### Newest Girls is given the 5 mai 2021 files max date

> remember to change this

hts_new_validate = hts_new[~hts_new.code.isin(hts_5Mai2021.code.to_list())]
hts_new_validate.code.count()

hts_new.merge(hts_5Mai2021, on='code', how='outer', suffixes=['', '_'], indicator=True)

a = hts_5Mai2021.code.to_list()
b = hts_new.code.to_list() 
len([elem for elem in b if elem not in a ])


In [96]:
hts_new_communes = []
for columns in hts_new.actual_commune.unique():
    globals()[f"hts_new_{name_handler(columns)}"] = hts_new[hts_new.actual_commune == columns]
    hts_new_communes.append(globals().get(f"hts_new_validate_{name_handler(columns)}"))

In [97]:
new_communes_general = hts_new.actual_commune.unique().tolist()
hts_new_dict = {}
for columns, communes in zip(hts_new.actual_commune.unique(), new_communes_general):
    hts_new_dict[f"{name_handler(communes)}"] =  globals().get(f"hts_new_{name_handler(columns)}").code.count()
    
    
hts_new_df = pd.DataFrame.from_dict(hts_new_dict, orient="index", columns=None)
hts_new_df.columns = ["nombre_filles_pour_depistage_hts"]
hts_new_df

,nombre_filles_pour_depistage_hts
Dessalines,165
Kenscoff,730
Gressier,270
CapHaïtien,236
Milot,75
PlaineduNord,91
QuartierMorin,38
Limonade,111
Verrettes,537
Liancourt,28


In [98]:
liste_hts_new = pd.ExcelWriter("rapportage_liste_filles_hts_23Juin2021.xlsx",engine="xlsxwriter")
hts_new_df.to_excel(liste_hts_new,sheet_name="Total_filles_pr_commune",index=True)
hts_new.to_excel(liste_hts_new,sheet_name="liste_des_filles_global",index=False,na_rep="NULL")
for columns,communes in zip(hts_new.actual_commune.unique(), communes_names_General):
    globals().get(f"hts5Juin_{name_handler(columns)}").to_excel(liste_hts_new,sheet_name=f"{name_handler(communes)}",index=False,na_rep="NULL")
liste_hts_new.save()

In [99]:
hts_new.to_excel("verification.xlsx",index=False,na_rep="NULL")